In [15]:
# Import Modules
import pandas as pd
import numpy as np
import sqlite3
import pickle
import os
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
# Import Dataset
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date, owner_user_id, tags
FROM questions
WHERE creation_date > '2021-09-01'
AND creation_date < '2023-09-01';
'''
df = pd.read_sql_query(query, conn)
conn.close()

# Import Users Data
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date
FROM users;
'''
user_df = pd.read_sql_query(query, conn)
conn.close()

In [16]:
user_df = user_df.rename(columns = {'creation_date':'account_creation_date',
                                    'id': 'owner_user_id'})
df = df.dropna(subset=['owner_user_id'])
df['owner_user_id'] = df['owner_user_id'].astype(int).astype(str)
user_df['owner_user_id'] = user_df.owner_user_id.astype(str)
df_merge = pd.merge(df, user_df, on = 'owner_user_id', how = 'left')
df_merge

,id,creation_date,owner_user_id,tags,account_creation_date
0,69006423,2021-09-01 00:00:35.237,14087917,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re...",2020-08-11 15:14:12.437
1,69006426,2021-09-01 00:00:55.583,13091928,"[""r"", ""shiny""]",2020-03-20 01:17:22.550
2,69006431,2021-09-01 00:01:23.670,8110267,"[""java"", ""installation"", ""compilation"", ""proje...",2017-06-04 12:15:02.013
3,69006437,2021-09-01 00:03:18.750,2745485,"[""firebase"", ""nuxt.js"", ""firebase-hosting""]",2013-09-04 06:00:02.613
4,69006439,2021-09-01 00:03:36.930,12139975,"[""r"", ""data.table""]",2019-09-29 20:57:59.120
...,...,...,...,...,...
2682915,77019848,2023-08-31 23:54:30.057,1006272,"[""docker"", ""docker-compose"", ""owasp-dependency...",2011-10-21 00:44:45.587
2682916,77019849,2023-08-31 23:55:21.660,15216800,"[""microsoft-graph-api"", ""sharepoint-online""]",2021-02-15 22:29:56.837
2682917,77019852,2023-08-31 23:56:42.653,433202,"[""python"", ""numpy"", ""floating-point""]",2010-08-27 17:36:47.873
2682918,77019854,2023-08-31 23:57:28.633,2532775,"[""angular"", ""typescript"", ""progressive-web-app...",2013-06-28 17:18:15.807


In [17]:
# erase outliers
# Compute difference between post creation_date - user creation_date.
df_merge['month_diff'] = (pd.to_datetime(df_merge['creation_date']) - pd.to_datetime(df_merge['account_creation_date'])) / pd.Timedelta(days=30)
# There are some negative values in the raw data.
df_merge = df_merge[df_merge['month_diff'] >= 0]
df_merge['year_month'] = pd.to_datetime(df_merge['creation_date']).dt.to_period('M') # Monthly
df_merge['account_year_month'] = pd.to_datetime(df_merge['account_creation_date']).dt.to_period('M') # Monthly
df_merge

/tmp/ipykernel_1867836/3777837347.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['year_month'] = pd.to_datetime(df_merge['creation_date']).dt.to_period('M') # Monthly
/tmp/ipykernel_1867836/3777837347.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['account_year_month'] = pd.to_datetime(df_merge['account_creation_date']).dt.to_period('M') # Monthly


,id,creation_date,owner_user_id,tags,account_creation_date,month_diff,year_month,account_year_month
0,69006423,2021-09-01 00:00:35.237,14087917,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re...",2020-08-11 15:14:12.437,12.845518,2021-09,2020-08
1,69006426,2021-09-01 00:00:55.583,13091928,"[""r"", ""shiny""]",2020-03-20 01:17:22.550,17.664897,2021-09,2020-03
2,69006431,2021-09-01 00:01:23.670,8110267,"[""java"", ""installation"", ""compilation"", ""proje...",2017-06-04 12:15:02.013,51.649684,2021-09,2017-06
3,69006437,2021-09-01 00:03:18.750,2745485,"[""firebase"", ""nuxt.js"", ""firebase-hosting""]",2013-09-04 06:00:02.613,97.291742,2021-09,2013-09
4,69006439,2021-09-01 00:03:36.930,12139975,"[""r"", ""data.table""]",2019-09-29 20:57:59.120,23.404297,2021-09,2019-09
...,...,...,...,...,...,...,...,...
2682915,77019848,2023-08-31 23:54:30.057,1006272,"[""docker"", ""docker-compose"", ""owasp-dependency...",2011-10-21 00:44:45.587,144.432170,2023-08,2011-10
2682916,77019849,2023-08-31 23:55:21.660,15216800,"[""microsoft-graph-api"", ""sharepoint-online""]",2021-02-15 22:29:56.837,30.901977,2023-08,2021-02
2682917,77019852,2023-08-31 23:56:42.653,433202,"[""python"", ""numpy"", ""floating-point""]",2010-08-27 17:36:47.873,158.408794,2023-08,2010-08
2682918,77019854,2023-08-31 23:57:28.633,2532775,"[""angular"", ""typescript"", ""progressive-web-app...",2013-06-28 17:18:15.807,123.875908,2023-08,2013-06


In [18]:
df_new = df_merge[df_merge['account_year_month'] >= '2020-09-01']
df_old = df_merge[df_merge['account_year_month'] < '2020-09-01']

In [21]:
df_new

,id,creation_date,owner_user_id,tags,account_creation_date,month_diff,year_month,account_year_month
5,69006440,2021-09-01 00:03:38.323,14306547,"[""swift""]",2020-09-19 17:10:42.167,11.542892,2021-09,2020-09
6,69006444,2021-09-01 00:04:08.040,15349492,"[""python"", ""discord.py"", ""text-to-speech""]",2021-03-07 19:22:00.957,5.906531,2021-09,2021-03
11,69006458,2021-09-01 00:06:28.170,16354132,"[""r"", ""matrix"", ""plot""]",2021-06-30 23:55:51.950,2.066912,2021-09,2021-06
13,69006471,2021-09-01 00:09:05.650,16620038,"[""go"", ""visual-studio-code""]",2021-08-08 19:57:07.367,0.772499,2021-09,2021-08
14,69006473,2021-09-01 00:09:29.763,16472582,"[""python"", ""interface"", ""getter-setter"", ""type...",2021-07-18 05:12:12.433,1.492993,2021-09,2021-07
...,...,...,...,...,...,...,...,...
2682909,77019829,2023-08-31 23:47:30.177,19170039,"[""blazor"", ""blazor-webassembly""]",2022-05-21 19:44:01.723,15.572303,2023-08,2022-05
2682911,77019837,2023-08-31 23:49:41.960,17610195,"[""pytorch""]",2021-12-07 04:49:11.880,21.093067,2023-08,2021-12
2682912,77019841,2023-08-31 23:52:15.803,22480648,"[""pine-script"", ""pine-script-v5"", ""stock"", ""tr...",2023-08-31 23:40:40.327,0.000268,2023-08,2023-08
2682913,77019842,2023-08-31 23:52:20.293,22480641,"[""excel"", ""vba"", ""excel-2016""]",2023-08-31 23:37:20.973,0.000347,2023-08,2023-08


In [25]:
len(df_new)+len(df_old)

2682779

In [24]:
#df_new.to_csv('df_isNew_new_vr2.csv', index = False)
#df_old.to_csv('df_isNew_old_vr2.csv', index = False)

### Calculate All Keys

#### 1) New Users

In [27]:
import math
import numpy as np

def calculate_entropy(probabilities):
    """ Calculate the Shannon entropy of a given list of probabilities. """
    entropy = 0
    for p in probabilities:
        if p > 0:
            entropy += p * math.log(p, 2)
    return -entropy

In [29]:
import pickle
with open(file = 'tagShare_new_vr2.pickle', mode = 'rb') as file:
    all_keys = pickle.load(file)
all_keys

,index,tag,tag_2021_09_01,tagShare_2021_09_01,tag_2021_09_02,tagShare_2021_09_02,tag_2021_09_03,tagShare_2021_09_03,tag_2021_09_04,tagShare_2021_09_04,...,tag_2023_08_27,tagShare_2023_08_27,tag_2023_08_28,tagShare_2023_08_28,tag_2023_08_29,tagShare_2023_08_29,tag_2023_08_30,tagShare_2023_08_30,tag_2023_08_31,tagShare_2023_08_31
0,swift,13371,25.0,0.472500,18.0,0.316456,13.0,0.283533,15.0,0.443262,...,11.0,0.323054,20.0,0.355619,20.0,0.344116,21.0,0.332963,20.0,0.327118
1,python,246799,335.0,6.331506,398.0,6.997187,289.0,6.303162,219.0,6.471631,...,174.0,5.110132,287.0,5.103129,275.0,4.731590,294.0,4.661487,273.0,4.465162
2,discordDOTpy,4566,12.0,0.226800,9.0,0.158228,9.0,0.196292,2.0,0.059102,...,2.0,0.058737,6.0,0.106686,5.0,0.086029,3.0,0.047566,6.0,0.098135
3,texttospeech,284,2.0,0.037800,1.0,0.017581,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.017206,NaN,NaN,2.0,0.032712
4,r,47444,58.0,1.096201,81.0,1.424051,61.0,1.330425,32.0,0.945626,...,19.0,0.558003,26.0,0.462304,42.0,0.722643,55.0,0.872047,46.0,0.752372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41185,plasmo,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.016356
41186,mscapi,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.016356
41187,nbitcoin,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.016356
41188,useswr,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.016356


In [30]:
# Measure score
entropy_Score = []
# Calculate Entropy for each monthly tag share column.
for i in range(3, all_keys.shape[1], 2):
    arr = np.array(all_keys.iloc[:, i])
    arr = arr/100
    arrList = arr.tolist()
    entropy_Score.append(calculate_entropy(arrList))

In [35]:
post = pd.read_csv("/data1/StackOverflow/diff_in_diff/numPosts_did.csv")
df_new = post[['year_month_day', 'T_d', 'P_t']]
df_new['entropy'] = entropy_Score
df_new['month'] = df_new['year_month_day'].str[5:7].astype(int).astype(str)
df_new['ln_entropy'] = np.log(df_new['entropy'])
df_new

/tmp/ipykernel_1867836/1018252532.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['ln_entropy'] = np.log(df_new['entropy'])


,year_month_day,T_d,P_t,entropy,month,ln_entropy
0,2021-09-01,0,0,8.940674,9,2.190611
1,2021-09-02,0,0,8.933225,9,2.189777
2,2021-09-03,0,0,8.921938,9,2.188513
3,2021-09-04,0,0,8.578207,9,2.149225
4,2021-09-05,0,0,8.441587,9,2.133170
...,...,...,...,...,...,...
725,2023-08-27,1,1,9.108446,8,2.209202
726,2023-08-28,1,1,9.634171,8,2.265316
727,2023-08-29,1,1,9.638980,8,2.265815
728,2023-08-30,1,1,9.702986,8,2.272434


In [40]:
new_Q = pd.read_csv("df_isNew_new_vr2.csv")
new_Q['year_month_day'] = pd.to_datetime(new_Q['creation_date']).dt.to_period('D').astype(str)
new_Q = new_Q.groupby('year_month_day')['id'].count().reset_index(name = 'count_q')
new_Q

,year_month_day,count_q
0,2021-09-01,1796
1,2021-09-02,1906
2,2021-09-03,1557
3,2021-09-04,1107
4,2021-09-05,1065
...,...,...
725,2023-08-27,1092
726,2023-08-28,1796
727,2023-08-29,1863
728,2023-08-30,2036


In [41]:
df_new['count_q'] = new_Q['count_q']
df_new

/tmp/ipykernel_1867836/2378691020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['count_q'] = new_Q['count_q']


,year_month_day,T_d,P_t,entropy,month,ln_entropy,count_q
0,2021-09-01,0,0,8.940674,9,2.190611,1796
1,2021-09-02,0,0,8.933225,9,2.189777,1906
2,2021-09-03,0,0,8.921938,9,2.188513,1557
3,2021-09-04,0,0,8.578207,9,2.149225,1107
4,2021-09-05,0,0,8.441587,9,2.133170,1065
...,...,...,...,...,...,...,...
725,2023-08-27,1,1,9.108446,8,2.209202,1092
726,2023-08-28,1,1,9.634171,8,2.265316,1796
727,2023-08-29,1,1,9.638980,8,2.265815,1863
728,2023-08-30,1,1,9.702986,8,2.272434,2036


In [1]:
# add answers data
# Import Modules
import pandas as pd
import numpy as np
import sqlite3
import pickle
import os
import matplotlib.pyplot as plt

# Import Dataset
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date, owner_user_id
FROM answers
WHERE creation_date > '2021-09-01'
AND creation_date < '2023-09-01';
'''
ans_df = pd.read_sql_query(query, conn)
conn.close()

# Import Users Data
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date
FROM users;
'''
user_df = pd.read_sql_query(query, conn)
conn.close()
user_df = user_df.rename(columns = {'creation_date':'account_creation_date',
                                    'id': 'owner_user_id'})
ans_df = ans_df.dropna(subset = 'owner_user_id')
ans_df['owner_user_id'] = ans_df['owner_user_id'].astype(int).astype(str)
# Merge answers to the correspond users
user_df['owner_user_id'] = user_df['owner_user_id'].astype(str)
ans_df = pd.merge(ans_df, user_df, on = 'owner_user_id', how = 'left')
ans_df

,id,creation_date,owner_user_id,account_creation_date
0,69006420,2021-09-01 00:00:18.070,7332046,2016-12-22 19:33:51.100
1,69006421,2021-09-01 00:00:28.823,14471093,2020-10-18 06:29:51.557
2,69006422,2021-09-01 00:00:35.537,523612,2010-11-29 07:44:17.673
3,69006429,2021-09-01 00:01:09.140,11269158,2019-03-27 23:15:22.047
4,69006430,2021-09-01 00:01:17.920,2096113,2013-02-21 15:43:28.953
...,...,...,...,...
3102831,77019850,2023-08-31 23:56:07.187,313768,2010-04-11 04:47:56.083
3102832,77019851,2023-08-31 23:56:19.123,13296852,2020-04-12 21:26:44.880
3102833,77019853,2023-08-31 23:56:47.647,5103949,2015-07-10 17:53:49.643
3102834,77019855,2023-08-31 23:57:36.797,16844882,2021-09-06 16:29:07.503


In [2]:
# Compute difference between post creation_date - user creation_date.
ans_df['month_diff'] = (pd.to_datetime(ans_df['creation_date']) - pd.to_datetime(ans_df['account_creation_date'])) / pd.Timedelta(days=30)
ans_df = ans_df[ans_df['month_diff'] >= 0]
ans_df['year_month_day'] = pd.to_datetime(ans_df['creation_date']).dt.to_period('D').astype(str)

/tmp/ipykernel_2084711/1277189143.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans_df['year_month_day'] = pd.to_datetime(ans_df['creation_date']).dt.to_period('D').astype(str)


In [3]:
ans_df['account_year_month'] = pd.to_datetime(ans_df['account_creation_date']).dt.to_period('M')
df_new_a = ans_df[ans_df['account_year_month'] >= '2020-09-01']
df_old_a = ans_df[ans_df['account_year_month'] < '2020-09-01']

/tmp/ipykernel_2084711/267750255.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans_df['account_year_month'] = pd.to_datetime(ans_df['account_creation_date']).dt.to_period('M')


In [50]:
len(df_new_a)

822701

In [51]:
len(df_old_a)

2280115

In [4]:
new_a = df_new_a.groupby('year_month_day')['id'].count().reset_index(name = 'count_a')
old_a = df_old_a.groupby('year_month_day')['id'].count().reset_index(name = 'count_a')

In [53]:
df_new['count_a'] = new_a['count_a']
df_new

,year_month_day,T_d,P_t,entropy,month,ln_entropy,count_q,count_a
0,2021-09-01,0,0,8.940674,9,2.190611,1796,1229
1,2021-09-02,0,0,8.933225,9,2.189777,1906,1234
2,2021-09-03,0,0,8.921938,9,2.188513,1557,1155
3,2021-09-04,0,0,8.578207,9,2.149225,1107,731
4,2021-09-05,0,0,8.441587,9,2.133170,1065,732
...,...,...,...,...,...,...,...,...
725,2023-08-27,1,1,9.108446,8,2.209202,1092,573
726,2023-08-28,1,1,9.634171,8,2.265316,1796,1002
727,2023-08-29,1,1,9.638980,8,2.265815,1863,975
728,2023-08-30,1,1,9.702986,8,2.272434,2036,956


#### 2) Old Users

In [54]:
import pickle
with open(file = 'tagShare_old_vr2.pickle', mode = 'rb') as file:
    all_keys = pickle.load(file)
all_keys
# Measure score
entropy_Score = []
# Calculate Entropy for each monthly tag share column.
for i in range(3, all_keys.shape[1], 2):
    arr = np.array(all_keys.iloc[:, i])
    arr = arr/100
    arrList = arr.tolist()
    entropy_Score.append(calculate_entropy(arrList))

In [55]:
df_old = post[['year_month_day', 'T_d', 'P_t']]
df_old['entropy'] = entropy_Score
df_old['month'] = df_old['year_month_day'].str[5:7].astype(int).astype(str)
df_old['ln_entropy'] = np.log(df_old['entropy'])
df_old

/tmp/ipykernel_1867836/385655014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_old['entropy'] = entropy_Score
/tmp/ipykernel_1867836/385655014.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_old['month'] = df_old['year_month_day'].str[5:7].astype(int).astype(str)
/tmp/ipykernel_1867836/385655014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

,year_month_day,T_d,P_t,entropy,month,ln_entropy
0,2021-09-01,0,0,9.646712,9,2.266617
1,2021-09-02,0,0,9.618467,9,2.263685
2,2021-09-03,0,0,9.655290,9,2.267506
3,2021-09-04,0,0,9.124537,9,2.210967
4,2021-09-05,0,0,9.201563,9,2.219373
...,...,...,...,...,...,...
725,2023-08-27,1,1,9.275680,8,2.227396
726,2023-08-28,1,1,9.723680,8,2.274564
727,2023-08-29,1,1,9.811312,8,2.283536
728,2023-08-30,1,1,9.804274,8,2.282818


In [58]:
old_Q = pd.read_csv("df_isNew_old_vr2.csv")
old_Q['year_month_day'] = pd.to_datetime(old_Q['creation_date']).dt.to_period('D').astype(str)
old_Q = old_Q.groupby('year_month_day')['id'].count().reset_index(name = 'count_q')
df_old['count_q'] = old_Q['count_q']
df_old

/tmp/ipykernel_1867836/2873965305.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_old['count_q'] = old_Q['count_q']


,year_month_day,T_d,P_t,entropy,month,ln_entropy,count_q
0,2021-09-01,0,0,9.646712,9,2.266617,2881
1,2021-09-02,0,0,9.618467,9,2.263685,2858
2,2021-09-03,0,0,9.655290,9,2.267506,2520
3,2021-09-04,0,0,9.124537,9,2.210967,1287
4,2021-09-05,0,0,9.201563,9,2.219373,1320
...,...,...,...,...,...,...,...
725,2023-08-27,1,1,9.275680,8,2.227396,784
726,2023-08-28,1,1,9.723680,8,2.274564,1487
727,2023-08-29,1,1,9.811312,8,2.283536,1689
728,2023-08-30,1,1,9.804274,8,2.282818,1737


In [59]:
df_old['count_a'] = old_a['count_a']
df_old

,year_month_day,T_d,P_t,entropy,month,ln_entropy,count_q,count_a
0,2021-09-01,0,0,9.646712,9,2.266617,2881,1229
1,2021-09-02,0,0,9.618467,9,2.263685,2858,1234
2,2021-09-03,0,0,9.655290,9,2.267506,2520,1155
3,2021-09-04,0,0,9.124537,9,2.210967,1287,731
4,2021-09-05,0,0,9.201563,9,2.219373,1320,732
...,...,...,...,...,...,...,...,...
725,2023-08-27,1,1,9.275680,8,2.227396,784,573
726,2023-08-28,1,1,9.723680,8,2.274564,1487,1002
727,2023-08-29,1,1,9.811312,8,2.283536,1689,975
728,2023-08-30,1,1,9.804274,8,2.282818,1737,956


In [60]:
df_old['ln_q'] = np.log(df_old['count_q'])
df_old['ln_a'] = np.log(df_old['count_a'])
df_new['ln_q'] = np.log(df_new['count_q'])
df_new['ln_a'] = np.log(df_new['count_a'])

In [62]:
df_old.describe()

,T_d,P_t,entropy,ln_entropy,count_q,count_a,ln_q,ln_a
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,0.500000,0.753425,9.572999,2.258681,1874.575342,1126.987671,7.477396,6.988640
std,0.500343,0.431313,0.219411,0.023111,589.832199,295.133438,0.360017,0.288327
min,0.000000,0.000000,8.976820,2.194646,691.000000,489.000000,6.538140,6.192362
25%,0.000000,1.000000,9.364821,2.236960,1371.000000,902.750000,7.223296,6.805445
50%,0.500000,1.000000,9.642440,2.266174,2010.000000,1141.000000,7.605889,7.039660
75%,1.000000,1.000000,9.738782,2.276116,2361.750000,1388.500000,7.767158,7.235979
max,1.000000,1.000000,9.952131,2.297787,2965.000000,1711.000000,7.994632,7.444833


In [63]:
df_new.describe()

,T_d,P_t,entropy,ln_entropy,count_q,count_a,ln_q,ln_a
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,0.500000,0.753425,9.148584,2.212985,1800.464384,1126.987671,7.458117,6.988640
std,0.500343,0.431313,0.320461,0.035081,474.275735,295.133438,0.282178,0.288327
min,0.000000,0.000000,8.411721,2.129626,845.000000,489.000000,6.739337,6.192362
25%,0.000000,1.000000,8.964294,2.193249,1409.500000,902.750000,7.250986,6.805445
50%,0.500000,1.000000,9.095804,2.207813,1826.000000,1141.000000,7.509882,7.039660
75%,1.000000,1.000000,9.461383,2.247219,2122.750000,1388.500000,7.660468,7.235979
max,1.000000,1.000000,9.747518,2.277013,3030.000000,1711.000000,8.016318,7.444833


In [64]:
df_old.to_csv("df_old_vr2.csv", index = False)
df_new.to_csv("df_new_vr2.csv", index = False)